<a href="https://colab.research.google.com/github/dtan6/Exercise-Classification-and-Recommendation-System-/blob/main/Keras_Exercise_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
main_folder_path = "/content/drive/MyDrive/ExerciseImages"
image_arrays = []
data_with_label = []
temp= []
image_size = (224, 224)

for i, exercise_folder in enumerate(os.listdir(main_folder_path)):
    exercise_folder_path = os.path.join(main_folder_path, exercise_folder)
    exercise_files = sorted(os.listdir(exercise_folder_path))

    for file in exercise_files:
        image = Image.open(os.path.join(exercise_folder_path, file))
        image = image.resize(image_size)
        image = image.convert("RGB")
        image_array = np.array(image)
        data_with_label.append([image_array, exercise_folder])
        temp.append([image_array, exercise_folder])

In [ ]:
unique_labels = set()
for _, label in temp:
    unique_labels.add(label)

print("Unique Labels:", unique_labels)

unique_labels1 = set()
for _, label in data_with_label:
    unique_labels1.add(label)

print("Unique Labels:", unique_labels1)



Unique Labels: {'chest fly machine', 'tricep dips', 'lat pulldown', 'bench press', 'incline bench press', 'leg extension', 't bar row', 'leg raises', 'barbell biceps curl', 'tricep pushdown', 'deadlift', 'decline bench press', 'pull up', 'hammer curl', 'plank', 'push up', 'lateral raises', 'hip thrust', 'romanian deadlift', 'russian twist', 'shoulder press', 'squat'}
Unique Labels: {'chest fly machine', 'tricep dips', 'lat pulldown', 'bench press', 'incline bench press', 'leg extension', 't bar row', 'leg raises', 'barbell biceps curl', 'tricep pushdown', 'deadlift', 'decline bench press', 'pull up', 'hammer curl', 'plank', 'push up', 'lateral raises', 'hip thrust', 'romanian deadlift', 'russian twist', 'shoulder press', 'squat'}


In [ ]:

np.random.shuffle(data_with_label)

random_state = 42
image_arrays_train, image_arrays_test = train_test_split(data_with_label, test_size=0.2, random_state=random_state)




In [ ]:
from sklearn.preprocessing import LabelEncoder

exercise_names = [exercise_name for _, exercise_name in data_with_label]

label_encoder = LabelEncoder()
label_encoder.fit(exercise_names)


X_train = [image_array for image_array, _ in image_arrays_train]
y_train = [exercise_name for _, exercise_name in image_arrays_train]
y_train_encoded = label_encoder.transform(y_train)

X_train = np.array(X_train)
y_train = to_categorical(y_train_encoded)

X_test = [image_array for image_array, _ in image_arrays_test]
y_test = [exercise_name for _, exercise_name in image_arrays_test]

y_test_encoded = label_encoder.transform(y_test)

X_test = np.array(X_test)
y_test = to_categorical(y_test_encoded)






In [ ]:
print(y_train[0])
print(image_arrays_train[5][1])
print(y_train[5])
print(y_test[0])
print(image_arrays_test[5][1])
print(y_test[5])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
incline bench press
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
pull up
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# Resnet50
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(len(os.listdir(main_folder_path)), activation='softmax'))


94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

batch_size = 32
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
347/347 [==============================] - 170s 338ms/step - loss: 0.5298 - accuracy: 0.8609 - val_loss: 1.7252 - val_accuracy: 0.6557
Epoch 2/10
347/347 [==============================] - 114s 329ms/step - loss: 0.1471 - accuracy: 0.9596 - val_loss: 2.4659 - val_accuracy: 0.4262
Epoch 3/10
347/347 [==============================] - 117s 337ms/step - loss: 0.1001 - accuracy: 0.9714 - val_loss: 0.3564 - val_accuracy: 0.9022
Epoch 4/10
347/347 [==============================] - 114s 329ms/step - loss: 0.0542 - accuracy: 0.9838 - val_loss: 1.2162 - val_accuracy: 0.6943
Epoch 5/10
347/347 [==============================] - 117s 336ms/step - loss: 0.0688 - accuracy: 0.9789 - val_loss: 6.1024 - val_accuracy: 0.3822
Epoch 6/10
347/347 [==============================] - 117s 336ms/step - loss: 0.0486 - accuracy: 0.9866 - val_loss: 0.5043 - val_accuracy: 0.8625
Epoch 7/10
347/347 [==============================] - 117s 336ms/step - loss: 0.0305 - accuracy: 0.9915 - val_loss: 1.0109 -

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

87/87 [==============================] - 8s 92ms/step - loss: 0.1412 - accuracy: 0.9697
Test loss: 0.1412, Test accuracy: 0.9697


In [ ]:

model.save('/content/drive/MyDrive/Saved Models/resnet50final.hdf5')

In [ ]:
from keras.models import load_model

In [ ]:
from keras.applications import VGG16
from keras.optimizers import Adam

In [ ]:
# VGG16
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model2 = Sequential()
model2.add(vgg16_model)

model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(len(os.listdir(main_folder_path)), activation='softmax'))

model2.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:

batch_size = 32
epochs = 10

model2.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
347/347 [==============================] - 181s 462ms/step - loss: 8.4805 - accuracy: 0.2020 - val_loss: 1.8959 - val_accuracy: 0.4626
Epoch 2/10
347/347 [==============================] - 158s 455ms/step - loss: 1.3193 - accuracy: 0.6394 - val_loss: 0.8966 - val_accuracy: 0.7708
Epoch 3/10
347/347 [==============================] - 148s 427ms/step - loss: 0.5551 - accuracy: 0.8672 - val_loss: 0.5516 - val_accuracy: 0.8881
Epoch 4/10
347/347 [==============================] - 158s 454ms/step - loss: 0.2878 - accuracy: 0.9312 - val_loss: 0.4337 - val_accuracy: 0.9123
Epoch 5/10
347/347 [==============================] - 158s 455ms/step - loss: 0.1640 - accuracy: 0.9582 - val_loss: 0.4778 - val_accuracy: 0.9307
Epoch 6/10
347/347 [==============================] - 158s 454ms/step - loss: 0.1358 - accuracy: 0.9632 - val_loss: 0.3668 - val_accuracy: 0.9437
Epoch 7/10
347/347 [==============================] - 158s 454ms/step - loss: 0.0917 - accuracy: 0.9737 - val_loss: 0.3745 -

In [ ]:
loss, accuracy = model2.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

87/87 [==============================] - 10s 118ms/step - loss: 0.3772 - accuracy: 0.9307
Test loss: 0.3772, Test accuracy: 0.9307


In [ ]:

model2.save('/content/drive/MyDrive/Saved Models/vgg16_final2.hdf5')

In [ ]:
# VGG19
from keras.applications import VGG19
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model3 = Sequential()
model3.add(vgg19_model)

model3.add(Flatten())
model3.add(Dense(256, activation='relu'))
model3.add(Dense(len(os.listdir(main_folder_path)), activation='softmax'))

model3.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 10
model3.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
347/347 [==============================] - 202s 521ms/step - loss: 4.3747 - accuracy: 0.1517 - val_loss: 2.2398 - val_accuracy: 0.3414
Epoch 2/10
347/347 [==============================] - 182s 526ms/step - loss: 1.3872 - accuracy: 0.6160 - val_loss: 0.9572 - val_accuracy: 0.7304
Epoch 3/10
347/347 [==============================] - 175s 504ms/step - loss: 0.5754 - accuracy: 0.8526 - val_loss: 0.5664 - val_accuracy: 0.8661
Epoch 4/10
347/347 [==============================] - 182s 525ms/step - loss: 0.3633 - accuracy: 0.9107 - val_loss: 0.4282 - val_accuracy: 0.9040
Epoch 5/10
347/347 [==============================] - 183s 526ms/step - loss: 0.2658 - accuracy: 0.9286 - val_loss: 0.3737 - val_accuracy: 0.9181
Epoch 6/10
347/347 [==============================] - 183s 527ms/step - loss: 0.1811 - accuracy: 0.9494 - val_loss: 0.3364 - val_accuracy: 0.9249
Epoch 7/10
347/347 [==============================] - 183s 526ms/step - loss: 0.1572 - accuracy: 0.9550 - val_loss: 0.3154 -

In [ ]:
loss, accuracy = model3.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

87/87 [==============================] - 12s 141ms/step - loss: 0.4703 - accuracy: 0.9491
Test loss: 0.4703, Test accuracy: 0.9491


In [ ]:
model3.save('/content/drive/MyDrive/Saved Models/vgg19_1_final1.h5')

In [ ]:
print(y_train.shape)

(11082, 22)


In [ ]:
!nvidia-smi

Sat Aug 19 05:24:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_flat, y_train)

RandomForestClassifier(random_state=42)

In [ ]:

y_pred = rf_model.predict(X_test_flat)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7610970768675568


In [ ]:
model_filename = '/content/drive/MyDrive/Saved Models/randomforest1.joblib'
joblib.dump(rf_model, model_filename)

['/content/drive/MyDrive/Saved Models/randomforest1.joblib']

In [ ]:
import joblib

In [ ]:

vgg16= load_model('/content/drive/MyDrive/Saved Models/vgg16_final2.hdf5')
vgg19= load_model('/content/drive/MyDrive/Saved Models/vgg19_1_final1.h5')
loaded_rf_model = joblib.load('/content/drive/MyDrive/Saved Models/randomforest1.joblib')


In [ ]:
resnet50= load_model('/content/drive/MyDrive/Saved Models/resnet50final.hdf5')

In [ ]:
resnet50_accuracy = resnet50.evaluate(X_test, y_test)[1]

resnet50_accuracy_df = pd.DataFrame({'Model': ['ResNet50'], 'Accuracy': [resnet50_accuracy]})

print(resnet50_accuracy_df)

87/87 [==============================] - 17s 96ms/step - loss: 0.0602 - accuracy: 0.9870
      Model  Accuracy
0  ResNet50  0.987008


In [ ]:
vgg16_accuracy = vgg16.evaluate(X_test, y_test)[1]

vgg16_accuracy_df = pd.DataFrame({'Model': ['VGG16'], 'Accuracy': [vgg16_accuracy]})

print(vgg16_accuracy_df)

87/87 [==============================] - 17s 143ms/step - loss: 0.0976 - accuracy: 0.9812
   Model  Accuracy
0  VGG16  0.981234


In [ ]:
vgg19_accuracy = vgg19.evaluate(X_test, y_test)[1]

vgg19_accuracy_df = pd.DataFrame({'Model': ['VGG19'], 'Accuracy': [vgg19_accuracy]})

print(vgg19_accuracy_df)

87/87 [==============================] - 12s 133ms/step - loss: 0.1782 - accuracy: 0.9574
   Model  Accuracy
0  VGG19  0.957416


In [ ]:
rf_accuracy = accuracy_score(y_test, loaded_rf_model.predict(X_test_flat))

rf_accuracy_df = pd.DataFrame({'Model': ['Random Forest'], 'Accuracy': [rf_accuracy]})

print(rf_accuracy_df)

           Model  Accuracy
0  Random Forest  0.758932


In [ ]:
from sklearn.metrics import classification_report

resnet50_predictions = resnet50.predict(X_test)
resnet50_predictions_single = np.argmax(resnet50_predictions, axis=1)

y_true_single = np.argmax(y_test, axis=1)

report = classification_report(y_true_single, resnet50_predictions_single, target_names=os.listdir(main_folder_path))

print(report)

87/87 [==============================] - 9s 89ms/step
                     precision    recall  f1-score   support

  romanian deadlift       1.00      1.00      1.00       120
              squat       0.95      0.98      0.97       128
            pull up       1.00      0.99      1.00       114
        tricep dips       0.99      0.99      0.99        97
            push up       0.99      0.97      0.98       103
          t bar row       0.99      0.99      0.99       100
     shoulder press       0.99      0.98      0.99       118
      russian twist       0.92      0.99      0.95       164
    tricep pushdown       1.00      0.99      1.00       126
              plank       0.99      0.99      0.99       173
        hammer curl       0.99      0.96      0.98       123
incline bench press       1.00      0.99      0.99        98
decline bench press       0.99      0.99      0.99       195
           deadlift       1.00      0.99      1.00       132
      leg extension       0.99

In [ ]:
from google.colab import files


In [ ]:
def preprocess_input(image):
    image_size = (224, 224)
    image = image.resize(image_size)
    image = image.convert("RGB")
    image_array = np.array(image)
    return image_array

In [ ]:

print("Unique Labels:", unique_labels)

print("Unique Labels1:", unique_labels1)



In [ ]:
input_images = []
num_sets_reps = {}

num_exercises = int(input("Enter the number of exercises: "))
for _ in range(num_exercises):
    uploaded = files.upload()

    uploaded_file = list(uploaded.keys())[0]

    num_sets = int(input("Enter the number of sets: "))
    num_reps = int(input("Enter the number of reps: "))

    input_image = Image.open(uploaded_file)
    input_image = preprocess_input(input_image)

    prediction = resnet50.predict(np.expand_dims(input_image, axis=0))
    exercise_names = ['barbell biceps curl','bench press','chest fly machine','deadlift','decline bench press','hammer curl','hip thrust','incline bench press','lat pulldown','lateral raise','leg extension','leg raise','plank','pull up','push up','romanian deadlift','russian twist','shoulder press','squat','t bar row','tricep dip','tricep pushdown']

    predicted_exercise_name = exercise_names[np.argmax(prediction)]

    input_images.append(input_image)
    if predicted_exercise_name in num_sets_reps:
      existing_tuple = num_sets_reps[predicted_exercise_name]
      new_num_sets = existing_tuple[0] + num_sets
      new_num_reps = existing_tuple[1] + num_reps
      num_sets_reps[predicted_exercise_name] = (new_num_sets, new_num_reps)
    else:
      num_sets_reps[predicted_exercise_name] = (num_sets, num_reps)

print(num_sets_reps)


Enter the number of exercises: 3


Saving pullup1.jpg to pullup1.jpg
Enter the number of sets: 1
Enter the number of reps: 10
1/1 [==============================] - 0s 23ms/step


Saving pullup2.jpg to pullup2.jpg
Enter the number of sets: 1
Enter the number of reps: 10
1/1 [==============================] - 0s 26ms/step


Saving pullup3.jpg to pullup3.jpg
Enter the number of sets: 1
Enter the number of reps: 10
1/1 [==============================] - 0s 22ms/step
{'shoulder press': (1, 10), 'incline bench press': (1, 10), 'pull up': (1, 10)}


In [ ]:
num_sets_reps

{'deadlift': (3, 10), 'pull up': (3, 10), 'tricep dip': (3, 10)}

In [ ]:
exercise_muscle_groups = {
    "barbell biceps curl": ["biceps", "forearms"],
    "bench press": ["chest", "triceps","shoulders"],
    "chest fly machine": ["chest", "back","shoulders"],
    "deadlift": ["core", "back","hamstrings","forearms"],
    "decline bench press": ["chest"],
    "hammer curl": ["biceps", "forearms"],
    "hip thrust": ["hamstrings"],
    "incline bench press": ["chest"],
    "lat pulldown": ["biceps", "back"],
    "lateral raise": ["back", "shoulders"],
    "leg extension": ["quadriceps"],
    "leg raise": ["back", "core", "hamstrings"],
    "plank": ["core"],
    "pull up": ["biceps", "back", "shoulders"],
    "push up": ["triceps", "core","chest","shoulders"],
    "romanian deadlift": ["hamstrings", "quadriceps"],
    "russian twist": ["core"],
    "shoulder press": ["shoulders", "triceps","chest"],
    "squat": ["quadriceps", "hamstrings"],
    "t bar row": ["back"],
    "tricep dip": ["triceps", "shoulders","chest"],
    "tricep pushdown": ["triceps"],

}

In [ ]:
muscle_group_frequency = {
    "biceps": 0,
    "forearms": 0,
    "chest": 0,
    "triceps": 0,
    "shoulders": 0,
    "core": 0,
    "back": 0,
    "hamstrings": 0,
    "quadriceps": 0
}

In [ ]:
for exercise, (num_sets, num_reps) in num_sets_reps.items():
    muscle_groups = exercise_muscle_groups.get(exercise, [])
    for muscle_group in muscle_groups:
        muscle_group_frequency[muscle_group] += (num_sets * num_reps)

In [ ]:
muscle_group_frequency

{'biceps': 30,
 'forearms': 30,
 'chest': 30,
 'triceps': 30,
 'shoulders': 60,
 'core': 30,
 'back': 60,
 'hamstrings': 30,
 'quadriceps': 0}

In [ ]:
total_muscle_frequency = sum(muscle_group_frequency.values())


most_worked_muscles = [muscle for muscle in muscle_group_frequency if muscle_group_frequency[muscle] == max(muscle_group_frequency.values())]
least_worked_muscles = [muscle for muscle in muscle_group_frequency if muscle_group_frequency[muscle] == min(muscle_group_frequency.values())]

underworked_muscles = [muscle for muscle in muscle_group_frequency if muscle != most_worked_muscles[0]]

underworked_exercise_recommendations = set()
for muscle in underworked_muscles:
    for exercise, muscle_groups in exercise_muscle_groups.items():
        if muscle in muscle_groups and exercise not in num_sets_reps:
            underworked_exercise_recommendations.add(exercise)

print("Most Worked Muscle Groups:", most_worked_muscles)
print("Least Worked Muscle Groups:", least_worked_muscles)
print("Underworked Muscle Groups:", underworked_muscles)
print("Exercise Recommendations for Underworked Muscle Groups:", list(underworked_exercise_recommendations))

Most Worked Muscle Groups: ['shoulders', 'back']
Least Worked Muscle Groups: ['quadriceps']
Underworked Muscle Groups: ['biceps', 'forearms', 'chest', 'triceps', 'core', 'back', 'hamstrings', 'quadriceps']
Exercise Recommendations for Underworked Muscle Groups: ['push up', 'leg extension', 'hammer curl', 'squat', 'bench press', 'decline bench press', 'hip thrust', 'barbell biceps curl', 'russian twist', 'lateral raise', 'lat pulldown', 'shoulder press', 'romanian deadlift', 'leg raise', 't bar row', 'chest fly machine', 'incline bench press', 'plank', 'tricep pushdown']


In [ ]:
import random
underworked_muscles = [muscle for muscle in underworked_muscles if muscle not in most_worked_muscles]

underworked_exercise_recommendations = set()
for muscle in underworked_muscles:
    eligible_exercises = [exercise for exercise, muscle_groups in exercise_muscle_groups.items() if muscle in muscle_groups and exercise not in num_sets_reps]
    if eligible_exercises:
        random_recommendations = random.sample(eligible_exercises, min(3, len(eligible_exercises)))
        underworked_exercise_recommendations.update(random_recommendations)
for exercise in num_sets_reps:
    if exercise in underworked_exercise_recommendations:
        underworked_exercise_recommendations.remove(exercise)
print("Most Worked Muscle Groups:", most_worked_muscles)
print("Muscle Groups You Should Target More:", underworked_muscles)
print()
print("Exercise Recommendations:")
for exercise in random.sample(underworked_exercise_recommendations, 3):
    print()
    print(f"{exercise}: Muscle Groups - {', '.join(exercise_muscle_groups[exercise])}")
    print()



Most Worked Muscle Groups: ['shoulders', 'back']
Muscle Groups You Should Target More: ['biceps', 'forearms', 'chest', 'triceps', 'core', 'hamstrings', 'quadriceps']

Exercise Recommendations:

shoulder press: Muscle Groups - shoulders, triceps, chest


push up: Muscle Groups - triceps, core, chest, shoulders


hip thrust: Muscle Groups - hamstrings



<ipython-input-18-99fe595d6390>:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for exercise in random.sample(underworked_exercise_recommendations, 3):
